In [26]:
"""
Manual calculation of the accommodation coefficient at 18 K, to verify script
"""

import matplotlib.pyplot as plt
import numpy as np
from gascooling.accommodation.evaluation.experiment import Connector
from gascooling.accommodation.evaluation.experiment import Helium, TestMass
from gascooling.accommodation.evaluation.experiment import InfluxDataContainer
from scipy import constants as const
from scipy.stats import linregress
from uncertainties import ufloat

## Experimental configurations

Low pressure regime indicates that any gas cooling is negligible. A pressure (warm) of less than 5e-6 mbar has been achieved. 

FMF pressure regime indicates that gas in the molecular flow cools the test mass. A pressure (warm) of around 1e-3 mbar (Kn ~ 10) has been achieved

High pressure regime indicates that viscous gas cooling dominates. A pressure (warm) of more than 1e-1 mbar has been achieved

- __L0__ : Low pressure, laser off. `P_screws = P_base_heat`
- __LL__ : Low pressure, laser on. `P_screws = P_base_heat + P_laser`
- __L0D__ : Low pressure, laser gets switched off, dynamic, time dependent cooldown is observed. ` P_screws + P_rad - P_base_heat ~ dQ/dt`

- __F0__: Kn around 10, laser off. `P_screws + P_FMF = P_base_heat`
- __FL__: Kn around 10, laser on. `P_screws + P_FMF = P_base_heat + P_laser`

- __H0__: High pressure, laser off. `P_visc + P_screws = P_base_heat `
- __HL__: High pressure, laser on. `P_visc + P_screws = P_base_heat + P_laser`
- __H0D__ : High pressure, laser gets switched off, dynamic, time dependent cooldown is observed. ` P_screws + P_rad + P_visc - P_base_heat ~ dQ/dt`



## Heating at base temperature (H0 config)

`P_visc` can be calculated. In the configuration of H0 we can estimate the P_screw cooling power.
If NO laser light is hitting the test mass, we have the following data (system in thermal equilibrium).

- Temperature TM : 7.39 K
- Temperature Frame: 7.36 K
- Temperature Steel : 8.55 K

In [15]:
number_of_screws = 6
thermal_conductivity_polyimide = 0.03
diameter_screws = 1e-3
distance_frame_tm = 2e-3
P_screws = (
    number_of_screws
    * thermal_conductivity_polyimide
    * np.pi
    * diameter_screws**2
    * (7.39 - 7.36)
    / (4 * distance_frame_tm)
)
print(f"{P_screws=}")
print(
    "P_visc = "
    + str(
        Helium(temperature=7.375).viscous_flow_cooling(
            temp_cold_surface=7.36,
            temp_hot_surface=7.39,
            distance=distance_frame_tm,
            surface=TestMass(temperature=7.39).surface_total,
        )
    )
)

With zero laser power, there is still a difference between Frame and Test mass, we assume a "discommoding base heating" P_base_heat, most is likely caused by the steel chamber (which in itself is heated by radiation from the heat shield (77 K))

So, at base temperature we assume P_visc + P_screws = P_base_heat.

Radiation is irrelevant because:


In [16]:
emissivity_steel_301 = 0.6
emissivity_tm = 0.85

P_steel = (
    const.Stefan_Boltzmann
    * emissivity_steel_301
    * emissivity_tm
    * 2
    * (2e-3 / 2) ** 2
    * np.pi
    * (8.625**4 - 7.453**4)
)
print(P_steel)

In [17]:
tm = TestMass(temperature=7.398)
helium = Helium(
    temperature=ufloat((7.398 + 7.36) / 2, 4e-3)
)  # 4e-3 is the given error of the temp sensors in this temp range
P_base_heat = (
    helium.viscous_flow_cooling(
        surface=tm.surface_total,
        distance=2e-3,
        temp_hot_surface=7.398,
        temp_cold_surface=7.36,
    )
    + P_screws
)
print(P_base_heat)

*We thus have a heating of the test mass at base temperature of 2.37 mW*. This needs to be substracted from the laser power

## Fitting photo diode voltage to absorbed laser power (HL config)

In our setup we are not able to directly measure P_laser or P_optical (see 2021-07-08-gas-cooling.pdf), but we split of roughly 1% of laser power and direct it onto a photodiode, which behaves linearly to light power.
We calibrate the photo voltage to absorbed light power in the test mass in regime HL.


We have the following data (and more):
```
pd voltage  |  tm temperature  |  frame temperature

-1.014      |  8.377           |  8.028
-1.173      |  8.516           |  8.124
-1.252      |  8.582           |  8.165
```

`P_absorbed_laser + P_base_heat = P_viscous + P_screws `

In [81]:
## Double check if screws are irrelevant
(
    number_of_screws
    * thermal_conductivity_polyimide
    * np.pi
    * diameter_screws**2
    * (8.582 - 8.165)
    / (4 * distance_frame_tm)
) / Helium(temperature=8.4).viscous_flow_cooling(
    temp_cold_surface=8.165,
    temp_hot_surface=8.582,
    distance=2e-3,
    surface=TestMass(temperature=8.582).surface_total,
)

Viscous cooling is dominating (screws, radiation insignificant), we thus can calculate the absorbed laser power with 

--> `P_absorbed_laser =  P_viscous - P_base_heat`

In [95]:
HL_measurements_metadata = Connector().pd_calibration_at_high_pressure()
P_absorbed_lasers = [0]
pd_voltages = [-0.001]
for metadata in HL_measurements_metadata:
    tm_data = InfluxDataContainer(
        start=metadata["Start"], stop=metadata["Stop"], field="Temperature_Testmass_(K)"
    ).data["Temperature_Testmass_(K)"]
    frame_data = InfluxDataContainer(
        start=metadata["Start"], stop=metadata["Stop"], field="Temperature_Holver_(K)"
    ).data["Temperature_Holver_(K)"]
    pd_data = InfluxDataContainer(
        start=metadata["Start"], stop=metadata["Stop"], field="Voltage_Photodiode_(V)"
    ).data["Voltage_Photodiode_(V)"]

    tm_temperature = tm_data.mean()
    frame_temperature = frame_data.mean()
    pd_voltage = pd_data.mean()

    helium = Helium(temperature=(tm_temperature + frame_temperature) / 2)
    viscous_cooling = helium.viscous_flow_cooling(
        surface=tm.surface_total,
        distance=2e-3,
        temp_hot_surface=tm_temperature,
        temp_cold_surface=frame_temperature,
    )
    print(f"{viscous_cooling=}")
    P_absorbed_laser_i = viscous_cooling - P_base_heat
    P_absorbed_lasers.append(P_absorbed_laser_i.nominal_value)
    pd_voltages.append(pd_voltage)

In [69]:
plt.scatter(pd_voltages, P_absorbed_lasers, marker="o")
plt.xlabel("Photo diode voltage (V)")
plt.ylabel("Absorbed Laser power (W)")

slope, intercept, _, _, _ = linregress(pd_voltages, P_absorbed_lasers)
x_for_fit = np.linspace(min(pd_voltages), max(pd_voltages))
y_for_fit = slope * x_for_fit + intercept
plt.plot(x_for_fit, y_for_fit)

## Discommoding (via screws and cables and radiation) cooling at temperatures > 10 K

### Experimental data
While the gas is in molecular flow, the other cooling channels might also have a significant role to the overall cooling.
At pressures below measurement range (so no gas cooling at all) we measured a tm temperature of 18 K with a pd voltage of 0.050 mV, while the frame is at 7.995 K.

This leads to a experimentally determined discommoding cooling power at 18 K:


In [85]:
P_discommoding_experimental = slope * -0.050 + intercept
print(P_discommoding_experimental)

1 mW of cooling power via screws and radiation

### Theory of discommoding
#### Conduction through screws

In [83]:
number_of_screws = 6
thermal_conductivity_polyimide = 0.03
diameter_screws = 1e-3
distance_frame_tm = 2e-3
P_screws = (
    number_of_screws
    * thermal_conductivity_polyimide
    * np.pi
    * diameter_screws**2
    * (18 - 7.995)
    / (4 * distance_frame_tm)
)
print(P_screws)

The screws have a conduction cooling of 0.7 mW

#### Radiation


In [84]:
emissivity_tm = 0.85
emissivity_frame = 0.05
P_radiative_discommoding = (
    const.Stefan_Boltzmann
    * emissivity_frame
    * emissivity_tm
    * TestMass(temperature=None).surface_total
    * (18**4 - 7.998**4)
)
print(f"Radiation cooling: {P_radiative_discommoding}")

print(
    f"Radiation+screws discommoding cooling theory {P_radiative_discommoding+P_screws}"
)

Supporting the experimental 1 mW of discommoding cooling at 18 K (cable for tm temperature sensor not calculated)

# Accommodation coefficient
In the regime of free molecular flow (FL), we have the following equation (p_viscous << P_fmf)
```
P_absorbed_laser + P_base_heat = P_discommoding + P_fmf
```

We measured the following
- TM temp 17.88 K( not quite 18 K, but close enough for now, just relevant for discommoding  )
- frame temp 8.148 K
- Pressure at gas handling 17.8e-4 mbar
- pd voltage -1.34 V

### Pressure correction



In [86]:
pressure_warm = 17.8e-4 * 100  # in Pa
pressure_in = 0.5 * pressure_warm * np.sqrt(8.148 / 293)
print(f"{pressure_in=} Pa")

### Absorbed laser power and FMF cooling power

In [115]:
P_absorbed_laser = slope * -1.34 + intercept
print(f"{P_absorbed_laser=} W")

P_fmf = P_absorbed_laser - P_discommoding_experimental
print(f"{P_fmf=} W")

### acc

In [116]:
alpha = (
    P_fmf
    * np.sqrt(np.pi * helium.mass * 8.148 / (8 * const.k))
    * 1
    / (pressure_in * tm.surface_total * (17.88 - 8.148))
)

In [117]:
print(f"alpha={round(alpha.nominal_value,4)} @ 17.88 K")
print("with")
print(f"P_fmf = {P_fmf*1000} mW")
print(f"P_base_heat = {P_base_heat*1000} mW")
print(f"P_absorbed_laser = {P_absorbed_laser*1000} mW")
print(f"P_discommoding_experimental = {P_discommoding_experimental*1000} mW")
print(f"")
print(f"{pressure_in=} Pa = {pressure_in/100*1e-4} * 10^-4 mbar")

# Verification with Christoph's formula


In [119]:
d = 2e-3
T_frame = 8.148
aP_opt_3 = absorbed_laser_power
aP_opt_2 = P_discommoding_experimental
alpha_cr = (
    np.sqrt(np.pi * helium.mass * T_frame / (8 * const.k))
    * (aP_opt_3 - aP_opt_2)
    / (pressure_in * TestMass(temperature=None).surface_total)
    / (17.88 - T_frame)
)
print(alpha_cr)